<a href="https://colab.research.google.com/github/1eetnals/coursera-tfcertificate/blob/main/C1_W4_happy_or_sad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**Things to Note:**
1. When coding the `class myCallback`, Python 3 will run into an error
```python
TypeError: '>' not supported between instances of 'NoneType' and 'float'
```
when using the code
```python
if(logs.get('accuracy')>0.99):
```

For Python 3, use the following equivalent code line

```python
if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
```

2. You can run the notebook using TensorFlow 2.5.0

In [ ]:
#!pip install tensorflow==2.5.0

In [ ]:
# this is used in downloading data from the google drive
#!pip install gdown

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

아래는 40개의 행복한 이미지, 40개의 슬픈 이미지를 포함하는 데이터셋에 대한 링크가 있는 코드.
이러한 이미지에서 100% 정확도로 훈련하는 컨볼루션 신경망을 만들어 .999의 적중 훈련 정확도보다 올라가면 훈련스탑.

힌트 - 3개의 컨볼루션 레이어에서 가장 잘 작동.

In [ ]:
import tensorflow as tf
import os
import zipfile

!gdown --id 1NvV6VhmrfU8JDZNoEbwJxwx_6dhyN5bf 

zip_ref = zipfile.ZipFile("./happy-or-sad.zip", 'r')  #zip파일 압축
zip_ref.extractall("./h-or-s")  #압축해제
zip_ref.close()

Downloading...
From: https://drive.google.com/uc?id=1NvV6VhmrfU8JDZNoEbwJxwx_6dhyN5bf
To: /content/happy-or-sad.zip
100% 2.67M/2.67M [00:00<00:00, 85.7MB/s]


In [ ]:
# GRADED FUNCTION: train_happy_sad_model
def train_happy_sad_model():
    # Please write your code only where you are indicated.
    # please do not remove # model fitting inline comments.

    DESIRED_ACCURACY = 0.999  #원하는 정확도

    class myCallback(tf.keras.callbacks.Callback): # YOUR CODE HERE):
        
        # YOUR CODE START HERE
        
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('accuracy') is not None and logs.get('accuracy') > DESIRED_ACCURACY:
                print("\nReached 99.9% accuracy so cancelling training!")
                self.model.stop_training = True
         
        
        # YOUR CODE END HERE
        
    callbacks = myCallback()
    
    # This Code Block should Define and Compile the Model. Please assume the images are 150 X 150 in your implementation.
    model = tf.keras.models.Sequential([
                                        
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)), # YOUR CODE HERE,
    #convolution filter 개수:16(output feature의 채널수), kernel_size(convolution filter):3x3
    tf.keras.layers.MaxPooling2D(2, 2), # YOUR CODE HERE,
    #pool_size, strides/ stride지정안해주면 풀링필터의 크기와 같음

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'), # YOUR CODE HERE,
    tf.keras.layers.MaxPooling2D(2,2), # YOUR CODE HERE,

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'), # YOUR CODE HERE
    tf.keras.layers.MaxPooling2D(2,2), # YOUR CODE HERE

    tf.keras.layers.Flatten(), # YOUR CODE HERE
    #17x17x32 -> 9248로
    tf.keras.layers.Dense(512, activation='relu'), # YOUR CODE HERE
    #9248 -> 512로
    tf.keras.layers.Dense(1, activation='sigmoid') # YOUR CODE HERE
    #512 -> 1로
    ])

    from tensorflow.keras.optimizers import RMSprop  #RMSprop: Adagrad에서 문제점을 개선하기 위해 Gt계산식에 지수이동평균 적용한것

    model.compile(loss='binary_crossentropy', # YOUR CODE HERE,
                  optimizer=RMSprop(learning_rate=0.001), # YOUR CODE HERE,
                  metrics=['accuracy']) # YOUR CODE HERE)
    
    # This code block should create an instance of an ImageDataGenerator called train_datagen 
    # And a train_generator by calling train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255) # YOUR CODE HERE

    # Please use a target_size of 150 X 150.
    train_generator = train_datagen.flow_from_directory( "./h-or-s", # YOUR CODE HERE   #flow_from_directory: Numpy Array Iterator 객체를 만들어줌
                                                        target_size=(150, 150), # YOUR CODE HERE
                                                        batch_size=10, # YOUR CODE HERE
                                                        class_mode='binary') # YOUR CODE HERE
    # Expected output: 'Found 80 images belonging to 2 classes'

    # This code block should call model.fit_generator and train for
    # a number of epochs.
    # model fitting
    history = model.fit(train_generator, # YOUR CODE HERE,
                        steps_per_epoch=8, # YOUR CODE HERE,
                        epochs=15, # YOUR CODE HERE,
                        verbose=1, # YOUR CODE HERE,   #0=slient, 1=progress bar, 2=one line per epoch: 프린트 되는 모양임(모델이 학습될때 뜨는)
                        callbacks=[callbacks] # YOUR CODE HERE,
                       )
    
    return history.history['accuracy'][-1]

In [ ]:
###이건 내가 그냥 짠거: 모델 층이 어떻게 변화되고, 파라미터가 변화되는지 확인하기 위해

model = tf.keras.models.Sequential([
                                        
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)), # YOUR CODE HERE,
    #convolution filter 개수:16(output feature의 채널수), kernel_size(convolution filter):3x3
    tf.keras.layers.MaxPooling2D(2, 2), # YOUR CODE HERE,
    #pool_size, strides

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'), # YOUR CODE HERE,
    tf.keras.layers.MaxPooling2D(2,2), # YOUR CODE HERE,

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'), # YOUR CODE HERE
    tf.keras.layers.MaxPooling2D(2,2), # YOUR CODE HERE

    tf.keras.layers.Flatten(), # YOUR CODE HERE
    tf.keras.layers.Dense(512, activation='relu'), # YOUR CODE HERE
    tf.keras.layers.Dense(1, activation='sigmoid') # YOUR CODE HERE
    ])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 74, 74, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 34, 34, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 17, 17, 32)      

### tf.keras.callbacks.Callback
새로운 Callback을 만들기 위해서는 tf.keras.callbacks.Callback을 상속해서 새로운 클래스를 만든다.

- on_epoch_begin(epoch, logs=None): 매번 epoch이 시작될 때 호출됨. epoch는 0으로 시작하는 현재 epoch의 index. 
- on_epoch_end(epoch, logs=None): 매번 epoch이 끝날 때 호출됨.logs['loss']는 현재 epoch 기준 loss가 저장됨. 이 외에 model.fit() 인자에 따라서 acc, val_loss 등도 저장됨.
- on_train_batch_begin(batch, logs=None): 매번 훈련 batch가 시작될 때 호출됨. batch는 0으로 시작하는 현재 batch의 index.
- on_train_batch_end(batch, logs=None): 매번 훈련 batch가 끝날 때 호출됨.
- on_train_begin(logs=None): 훈련이 시작될 때 호출됨.
- on_train_end(logs=None): 훈련이 끝날 때 호출됨.


In [ ]:
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Epoch 1/15
8/8 [==============================] - 14s 26ms/step - loss: 0.9356 - accuracy: 0.5250
Epoch 2/15
8/8 [==============================] - 0s 25ms/step - loss: 0.6259 - accuracy: 0.7125
Epoch 3/15
8/8 [==============================] - 0s 26ms/step - loss: 0.3463 - accuracy: 0.9000
Epoch 4/15
8/8 [==============================] - 0s 24ms/step - loss: 0.2008 - accuracy: 0.9625
Epoch 5/15
8/8 [==============================] - 0s 25ms/step - loss: 0.0881 - accuracy: 0.9625
Epoch 6/15
8/8 [==============================] - 0s 24ms/step - loss: 0.0867 - accuracy: 0.9750
Epoch 7/15
8/8 [==============================] - 0s 27ms/step - loss: 0.0454 - accuracy: 0.9750
Epoch 8/15
7/8 [=========================>....] - ETA: 0s - loss: 0.0184 - accuracy: 1.0000
Reached 99.9% accuracy so cancelling training!
8/8 [==============================] - 0s 27ms/step - loss: 0.0209 - accuracy: 1.0000


1.0